In [1]:
# Import the required modules
from pyesgf.search import SearchConnection
import os
import sys
import importlib
import pandas as pd
import requests
from tqdm import tqdm

# Set the os environment to on
os.environ['ESGF_PYCLIENT_NO_FACETS_STAR_WARNING'] = "on"

In [2]:
# Import the functions
sys.path.append('/home/users/benhutch/downloading-data/DAMIP/')

# Import the functions
from testing_download_functions import *

In [3]:
# Import the dictionaries
import dictionaries as dicts

In [4]:
# Import the functions again, so as to not have to restart the kernel
importlib.reload(sys.modules['testing_download_functions'])

# Import the functions again
from testing_download_functions import find_valid_nodes, create_results_list, \
                                        extract_file_context, check_file_exists_jasmin

# Import the dictionaries again, so as to not have to restart the kernel
importlib.reload(sys.modules['dictionaries'])

# Import the dictionaries again
import dictionaries as dicts

In [5]:
# Set up the search connection
connection = SearchConnection(dicts.search_connection, distrib=True)

In [6]:
# Set up the paramas
# To speed up the check, only search for year s1961
params = {
    'activity_id': 'DCPP',
    'experiment_id': 'dcppA-hindcast',
    'latest': True,
    'sub_experiment_id': 's1961',
    'project': 'CMIP6',
    'table_id': 'Amon',
}

In [7]:
# Loop over the variables
# variables = dicts.variables
models = dicts.models

# Set up the test variable - 'pr' precip
variables = ['pr']

# Create a dataframe for the results
# Containing three columns: model, experiment, does_model_exist
results_df = pd.DataFrame(columns=['model', 'variable', 'does_model_exist'])

# Loop over the variables
for variable in variables:
    print("Checking whether models exist for variable: " + variable)

    # Loop over the models
    for model in models:
        print("Checking whether model: " + model + " exists for variable: " + variable)

        # Set up the params
        params['variable_id'] = variable
        params['source_id'] = model

        # Query the database
        ctx = connection.new_context(**params)

        try:
            # Get the results from the query
            results = ctx.search()
        except:
            print("Model: " + model + " does not exist for variable: " + variable)
            results = []

        # If the length of results is greater than 0
        if len(results) > 0:
            print("Model: " + model + " exists for variable: " + variable)

            # Append True to the dataframe
            results_df = pd.concat([results_df, pd.DataFrame({'model': [model], 'variable': [variable], 'does_model_exist': [True]})], ignore_index=True)
        else:
            print("Model: " + model + " does not exist for variable: " + variable)

            # Append False to the dataframe
            results_df = pd.concat([results_df, pd.DataFrame({'model': [model], 'variable': [variable], 'does_model_exist': [False]})], ignore_index=True)

# Print the dataframe
results_df

Checking whether models exist for variable: pr
Checking whether model: BCC-CSM2-MR exists for variable: pr
Model: BCC-CSM2-MR exists for variable: pr
Checking whether model: MPI-ESM1-2-HR exists for variable: pr
Model: MPI-ESM1-2-HR exists for variable: pr
Checking whether model: CanESM5 exists for variable: pr
Model: CanESM5 exists for variable: pr
Checking whether model: CMCC-CM2-SR5 exists for variable: pr
Model: CMCC-CM2-SR5 exists for variable: pr
Checking whether model: HadGEM3-GC31-MM exists for variable: pr
Model: HadGEM3-GC31-MM exists for variable: pr
Checking whether model: EC-Earth3 exists for variable: pr
Model: EC-Earth3 exists for variable: pr
Checking whether model: MPI-ESM1-2-LR exists for variable: pr
Model: MPI-ESM1-2-LR exists for variable: pr
Checking whether model: FGOALS-f3-L exists for variable: pr
Model: FGOALS-f3-L exists for variable: pr
Checking whether model: MIROC6 exists for variable: pr
Model: MIROC6 exists for variable: pr
Checking whether model: IPSL-C

,model,variable,does_model_exist
0,BCC-CSM2-MR,pr,True
1,MPI-ESM1-2-HR,pr,True
2,CanESM5,pr,True
3,CMCC-CM2-SR5,pr,True
4,HadGEM3-GC31-MM,pr,True
5,EC-Earth3,pr,True
6,MPI-ESM1-2-LR,pr,True
7,FGOALS-f3-L,pr,True
8,MIROC6,pr,True
9,IPSL-CM6A-LR,pr,True


In [7]:
# Create an empty dictionary to store the valid nodes for each variable
variable_nodes = {}


# Find the valid nodes for each variable and models list combination
# Loop over the var models dictionary
for var, models_list in dicts.var_models_test_pr.items():
    print("Finding the valid nodes for variable: " + var)
    print("Models list: " + str(models_list))

    # Set up the variable_id for the params
    params['variable_id'] = var

    # Append the variable to the dictionary
    variable_nodes[var] = {}

    # Find the valid nodes
    valid_nodes = find_valid_nodes(params=params, 
                                   models_list=models_list,
                                   conn=connection)
    
    # Append the valid nodes to the dictionary
    variable_nodes[var] = valid_nodes


Finding the valid nodes for variable: pr
Models list: ['BCC-CSM2-MR', 'MPI-ESM1-2-HR', 'CanESM5', 'CMCC-CM2-SR5', 'HadGEM3-GC31-MM', 'EC-Earth3', 'MPI-ESM1-2-LR', 'FGOALS-f3-L', 'MIROC6', 'IPSL-CM6A-LR', 'NorCPM1']
trying to find valid nodes for model: BCC-CSM2-MR
{'activity_id': 'DCPP', 'experiment_id': 'dcppA-hindcast', 'latest': True, 'sub_experiment_id': 's1961', 'project': 'CMIP6', 'table_id': 'Amon', 'variable_id': 'pr', 'source_id': 'BCC-CSM2-MR'}
40
{'esgf.ceda.ac.uk', 'cmip.bcc.cma.cn', 'esgf-data1.llnl.gov', 'esgf3.dkrz.de', 'esgf.nci.org.au'}
trying to find valid files for node: esgf.ceda.ac.uk
8
trying to find valid files for node: cmip.bcc.cma.cn
8
trying to find valid files for node: esgf-data1.llnl.gov
8
trying to find valid files for node: esgf3.dkrz.de
8
trying to find valid files for node: esgf.nci.org.au
8
trying to find valid nodes for model: MPI-ESM1-2-HR
{'activity_id': 'DCPP', 'experiment_id': 'dcppA-hindcast', 'latest': True, 'sub_experiment_id': 's1961', 'proje

In [8]:
# For each variable, create the results list
# Initialize an empty dictionary to store the results
results = {}

# Loop over the variables
for var in dicts.variables_test_pr:
    print("Finding the results for variable: " + var)

    # Extract the valid nodes for the variable
    valid_nodes = variable_nodes[var]

    # Append the variable to params
    params['variable_id'] = var

    # print the type of valid nodes
    print("Valid nodes type: " + str(type(valid_nodes)))
    print("Valid nodes: " + str(valid_nodes))

    # Find the results for the variable
    var_results = create_results_list(params=params,
                                      max_results_list=valid_nodes,
                                      connection=connection)

    # Append the results to the dictionary
    results[var] = var_results

Finding the results for variable: pr
Valid nodes type: <class 'list'>
Valid nodes: [{'source_id': 'BCC-CSM2-MR', 'data_node': 'esgf.ceda.ac.uk', 'num_results': 8}, {'source_id': 'MPI-ESM1-2-HR', 'data_node': 'esgf3.dkrz.de', 'num_results': 10}, {'source_id': 'CanESM5', 'data_node': 'esgf.ceda.ac.uk', 'num_results': 40}, {'source_id': 'CMCC-CM2-SR5', 'data_node': 'esgf-data1.llnl.gov', 'num_results': 20}, {'source_id': 'HadGEM3-GC31-MM', 'data_node': 'esgf-data1.llnl.gov', 'num_results': 10}, {'source_id': 'EC-Earth3', 'data_node': 'esgf.bsc.es', 'num_results': 21}, {'source_id': 'MPI-ESM1-2-LR', 'data_node': 'esgf.dwd.de', 'num_results': 16}, {'source_id': 'FGOALS-f3-L', 'data_node': 'esgf-data1.llnl.gov', 'num_results': 9}, {'source_id': 'MIROC6', 'data_node': 'esgf-data02.diasjp.net', 'num_results': 10}, {'source_id': 'IPSL-CM6A-LR', 'data_node': 'esgf-data1.llnl.gov', 'num_results': 10}, {'source_id': 'NorCPM1', 'data_node': 'esgf-data1.llnl.gov', 'num_results': 20}]


  0%|          | 0/11 [00:00<?, ?it/s]

Querying for source_id: BCC-CSM2-MR and data_node: esgf.ceda.ac.uk


  9%|▉         | 1/11 [00:02<00:26,  2.66s/it]

Found 8 results.
Querying for source_id: MPI-ESM1-2-HR and data_node: esgf3.dkrz.de


...

Found 10 results.
Querying for source_id: IPSL-CM6A-LR and data_node: esgf-data1.llnl.gov


 91%|█████████ | 10/11 [00:25<00:02,  2.30s/it]

Found 10 results.
Querying for source_id: NorCPM1 and data_node: esgf-data1.llnl.gov


100%|██████████| 11/11 [00:26<00:00,  2.42s/it]

Found 20 results.


In [10]:
print("Results: " + str(results))
# print the length of the results
print("Length of results: " + str(len(results)))
print(type(results['pr']))

Results: {'pr': [<pyesgf.search.results.ResultSet object at 0x7f9684ebbd00>, <pyesgf.search.results.ResultSet object at 0x7f9685027a60>, <pyesgf.search.results.ResultSet object at 0x7f9685026dd0>, <pyesgf.search.results.ResultSet object at 0x7f9685027880>, <pyesgf.search.results.ResultSet object at 0x7f96850279a0>, <pyesgf.search.results.ResultSet object at 0x7f9684ebbdc0>, <pyesgf.search.results.ResultSet object at 0x7f9684ebbd90>, <pyesgf.search.results.ResultSet object at 0x7f96850277c0>, <pyesgf.search.results.ResultSet object at 0x7f9684f682b0>, <pyesgf.search.results.ResultSet object at 0x7f9684f69540>, <pyesgf.search.results.ResultSet object at 0x7f9684f6a830>]}
Length of results: 1
<class 'list'>


In [11]:
# Initialize an empty dictionary to store the results file context
results_file_context = {}

# For each of the variables
for var, results in results.items():
    print("Variable: " + var)
    print("Results: " + str(results))

    # Initialize an empty dictionary to store the file context
    file_context_list = []

    # Loop over the results
    for result in results:
        print("Result: " + str(result))

        # Extract the file context
        file_context = extract_file_context(result)

        # Append the file context to the list
        file_context_list.append(file_context)

    # Append the file context list to the dictionary
    results_file_context[var] = file_context_list

Variable: pr
Results: [<pyesgf.search.results.ResultSet object at 0x7f9684ebbd00>, <pyesgf.search.results.ResultSet object at 0x7f9685027a60>, <pyesgf.search.results.ResultSet object at 0x7f9685026dd0>, <pyesgf.search.results.ResultSet object at 0x7f9685027880>, <pyesgf.search.results.ResultSet object at 0x7f96850279a0>, <pyesgf.search.results.ResultSet object at 0x7f9684ebbdc0>, <pyesgf.search.results.ResultSet object at 0x7f9684ebbd90>, <pyesgf.search.results.ResultSet object at 0x7f96850277c0>, <pyesgf.search.results.ResultSet object at 0x7f9684f682b0>, <pyesgf.search.results.ResultSet object at 0x7f9684f69540>, <pyesgf.search.results.ResultSet object at 0x7f9684f6a830>]
Result: <pyesgf.search.results.ResultSet object at 0x7f9684ebbd00>
Extracting file context for 8 datasets...


 12%|█▎        | 1/8 [00:09<01:09,  9.93s/it]

Processed 1 out of 8 results.


 25%|██▌       | 2/8 [00:18<00:55,  9.24s/it]

Processed 2 out of 8 results.


...

Processed 18 out of 20 results.


 95%|█████████▌| 19/20 [00:27<00:01,  1.33s/it]

Processed 19 out of 20 results.


100%|██████████| 20/20 [00:28<00:00,  1.43s/it]

Processed 20 out of 20 results.


In [12]:
# Create an empty dataframe to store the results
df = pd.DataFrame()

for var, file_context_lists in results_file_context.items():
    print("Variable: " + var)

    # Loop over the file contexts
    for file_context in file_context_lists:
        # Convert the dictionary to a dataframe
        file_context_df = pd.DataFrame.from_dict(file_context)

        # Add a new column on the far left of the dataframe
        # containing the variable name
        file_context_df.insert(0, 'variable', var)

        # Concatenate the dataframe to the results dataframe
        df = pd.concat([df, file_context_df], ignore_index=True)

# Print the dataframe
df

Variable: pr


,variable,0,1,2,3,4,5,6,7,8,...,221,222,223,224,225,226,227,228,229,230
0,pr,{'filename': 'pr_Amon_BCC-CSM2-MR_dcppA-hindca...,{'filename': 'pr_Amon_BCC-CSM2-MR_dcppA-hindca...,{'filename': 'pr_Amon_BCC-CSM2-MR_dcppA-hindca...,{'filename': 'pr_Amon_BCC-CSM2-MR_dcppA-hindca...,{'filename': 'pr_Amon_BCC-CSM2-MR_dcppA-hindca...,{'filename': 'pr_Amon_BCC-CSM2-MR_dcppA-hindca...,{'filename': 'pr_Amon_BCC-CSM2-MR_dcppA-hindca...,{'filename': 'pr_Amon_BCC-CSM2-MR_dcppA-hindca...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,pr,None,None,None,None,None,None,None,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,pr,{'filename': 'pr_Amon_MPI-ESM1-2-HR_dcppA-hind...,{'filename': 'pr_Amon_MPI-ESM1-2-HR_dcppA-hind...,{'filename': 'pr_Amon_MPI-ESM1-2-HR_dcppA-hind...,{'filename': 'pr_Amon_MPI-ESM1-2-HR_dcppA-hind...,{'filename': 'pr_Amon_MPI-ESM1-2-HR_dcppA-hind...,{'filename': 'pr_Amon_MPI-ESM1-2-HR_dcppA-hind...,{'filename': 'pr_Amon_MPI-ESM1-2-HR_dcppA-hind...,{'filename': 'pr_Amon_MPI-ESM1-2-HR_dcppA-hind...,{'filename': 'pr_Amon_MPI-ESM1-2-HR_dcppA-hind...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,pr,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,pr,{'filename': 'pr_Amon_CanESM5_dcppA-hindcast_s...,{'filename': 'pr_Amon_CanESM5_dcppA-hindcast_s...,{'filename': 'pr_Amon_CanESM5_dcppA-hindcast_s...,{'filename': 'pr_Amon_CanESM5_dcppA-hindcast_s...,{'filename': 'pr_Amon_CanESM5_dcppA-hindcast_s...,{'filename': 'pr_Amon_CanESM5_dcppA-hindcast_s...,{'filename': 'pr_Amon_CanESM5_dcppA-hindcast_s...,{'filename': 'pr_Amon_CanESM5_dcppA-hindcast_s...,{'filename': 'pr_Amon_CanESM5_dcppA-hindcast_s...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,pr,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,pr,{'filename': 'pr_Amon_CMCC-CM2-SR5_dcppA-hindc...,{'filename': 'pr_Amon_CMCC-CM2-SR5_dcppA-hindc...,{'filename': 'pr_Amon_CMCC-CM2-SR5_dcppA-hindc...,{'filename': 'pr_Amon_CMCC-CM2-SR5_dcppA-hindc...,{'filename': 'pr_Amon_CMCC-CM2-SR5_dcppA-hindc...,{'filename': 'pr_Amon_CMCC-CM2-SR5_dcppA-hindc...,{'filename': 'pr_Amon_CMCC-CM2-SR5_dcppA-hindc...,{'filename': 'pr_Amon_CMCC-CM2-SR5_dcppA-hindc...,{'filename': 'pr_Amon_CMCC-CM2-SR5_dcppA-hindc...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,pr,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,pr,{'filename': 'pr_Amon_HadGEM3-GC31-MM_dcppA-hi...,{'filename': 'pr_Amon_HadGEM3-GC31-MM_dcppA-hi...,{'filename': 'pr_Amon_HadGEM3-GC31-MM_dcppA-hi...,{'filename': 'pr_Amon_HadGEM3-GC31-MM_dcppA-hi...,{'filename': 'pr_Amon_HadGEM3-GC31-MM_dcppA-hi...,{'filename': 'pr_Amon_HadGEM3-GC31-MM_dcppA-hi...,{'filename': 'pr_Amon_HadGEM3-GC31-MM_dcppA-hi...,{'filename': 'pr_Amon_HadGEM3-GC31-MM_dcppA-hi...,{'filename': 'pr_Amon_HadGEM3-GC31-MM_dcppA-hi...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,pr,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Set up the directory for the data on JASMIN
dcpp_dir_badc = "/badc/cmip6/data/CMIP6/DCPP/"

# Checkk whether these files exist on JASMIN
jasmin_files_df = check_file_exists_jasmin(df=df,
                                            directory=dcpp_dir_badc)

KeyError: 'filename'

In [64]:
# Now set up the group work space directory
dcpp_dir_gws = "/gws/nopw/j04/canari/users/benhutch/"

# Check whether these files exist on JAASMIN
jasmin_files_df_gws = check_file_exists_jasmin(df=df,
                                            directory=dcpp_dir_gws) 

BCC
gws
File does not exist for tas_Amon_BCC-CSM2-MR_dcppA-hindcast_s1961-r4i1p1f1_gn_196101-197012.nc
BCC
gws
File does not exist for tas_Amon_BCC-CSM2-MR_dcppA-hindcast_s1961-r2i1p1f1_gn_196101-197012.nc
BCC
gws
File does not exist for tas_Amon_BCC-CSM2-MR_dcppA-hindcast_s1961-r3i1p1f1_gn_196101-197012.nc
BCC
gws
File does not exist for tas_Amon_BCC-CSM2-MR_dcppA-hindcast_s1961-r5i1p1f1_gn_196101-197012.nc
BCC
gws
File does not exist for tas_Amon_BCC-CSM2-MR_dcppA-hindcast_s1961-r8i1p1f1_gn_196101-197012.nc
BCC
gws
File does not exist for tas_Amon_BCC-CSM2-MR_dcppA-hindcast_s1961-r6i1p1f1_gn_196101-197012.nc
BCC
gws
File does not exist for tas_Amon_BCC-CSM2-MR_dcppA-hindcast_s1961-r7i1p1f1_gn_196101-197012.nc
MPI-M
gws
File does not exist for tas_Amon_MPI-ESM1-2-HR_dcppA-hindcast_s1961-r3i1p1f1_gn_196111-197112.nc
MPI-M
gws
File does not exist for tas_Amon_MPI-ESM1-2-HR_dcppA-hindcast_s1961-r1i1p1f1_gn_196111-197112.nc
MPI-M
gws
File does not exist for tas_Amon_MPI-ESM1-2-HR_dcppA-hi